### 주정차 단속 데이터 + 고정형 cctv 위치 데이터 기반 시각화

In [2]:
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import branca.colormap as cm
from collections import defaultdict
import numpy as np
import pandas as pd

In [34]:
lat = 37.264523	
lng = 127.031856

In [3]:
illigal_fixture = pd.read_csv('주정차단속_고정형_위경도추가.csv')
illigal_fixture.head()

,집계년도,시군명,관리기관명,단속일시정보,단속장소,단속방법,데이터기준일자,단속일시_월,단속일시_연,단속일시_일,단속일시_요일,단속일시_시,단속일시_isdaytime,단속일시_season,latitude,longitude,cctv_name,fuzzy_matched_name
0,2022,수원시,장안구청,2022-01-01 08:11:00,하광교동 광교주차장입구,고정형,2023-12-01,1,2022,1,Saturday,8,morning,winter,37.300916,127.031054,광교주차장입구,광교주차장입구
1,2022,수원시,장안구청,2022-01-01 09:00:00,영화동 장안문거북시장,고정형,2023-12-01,1,2022,1,Saturday,9,morning,winter,37.291496,127.013938,장안문거북시장,장안문거북시장
2,2022,수원시,권선구청,2022-01-01 09:01:00,권선동 청학골프클럽,고정형,2023-12-01,1,2022,1,Saturday,9,morning,winter,37.260492,127.030071,청학골프클럽,청학골프클럽
3,2022,수원시,권선구청,2022-01-01 09:01:00,금곡동 리치빌딩 앞,고정형,2023-12-01,1,2022,1,Saturday,9,morning,winter,37.272862,126.943423,리치빌딩 앞,리치빌딩 앞
4,2022,수원시,장안구청,2022-01-01 09:01:00,하광교동 광교주차장입구,고정형,2023-12-01,1,2022,1,Saturday,9,morning,winter,37.300916,127.031054,광교주차장입구,광교주차장입구


In [19]:
illigal_fixture.groupby('cctv_name').size().sort_values(ascending=False)

cctv_name
CU 사거리                              11061
시드니볼링장 맞은편                           9849
무비사거리                                8870
팅스몰 주차장 출구 옆                         7765
탐앤탐스 사거리                             7387
                                    ...  
영통아이파크캐슬 1단지입구                          4
광교산더샵퍼스트 105동                           4
경기도 수원시 장안구 연무동 244-6 105동 옆 삼거리        3
금곡초등학교 입구                               1
명인중학교                                   1
Length: 448, dtype: int64

In [27]:
illigal_fixture_agg=illigal_fixture.groupby('cctv_name').agg(
    단속건수=('cctv_name', 'size'),
    평균위도=('latitude', 'mean'),
    평균경도=('longitude', 'mean')
)
illigal_fixture_agg

,단속건수,평균위도,평균경도
cctv_name,,,
(구)갤러리아백화점,742,37.261096,127.033348
CJ아파트 오거리,2047,37.276241,127.029439
CU 사거리,11061,37.266184,127.032565
E-마트 에브리데이 앞,1401,37.241577,127.058663
KBS드라마센터,1583,37.267498,127.031209
...,...,...,...
힐스테이트,209,37.241082,127.049583
힐스테이트 119동 ~방죽공원 삼거리,1037,37.244795,127.050789
힐스테이트 정문앞,74,37.287282,127.063896


In [28]:
def geo_transform(df):
    df['latitude'] = df['latitude'].astype(float)
    df['longitude'] = df['longitude'].astype(float)
    df['geometry'] = df.apply(lambda row: Point([row['longitude'],row['latitude']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry')
    df.crs = {'init':'epsg:4326'}
    df = df.to_crs({'init':'epsg:4326'})
    return df

In [30]:
illigal_fixture = geo_transform(illigal_fixture)

/opt/anaconda3/envs/data_venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/anaconda3/envs/data_venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [31]:
lat = 37.264523	
lng = 127.031856
m = folium.Map(location=[lat, lng], zoom_start=11, title='CCTV_Location')

for _, row in illigal_fixture.iterrows():
    folium.Circle(
        location=(row['latitude'], row['longitude']), 
        popup=row['cctv_name'], 
        radius=50, 
        color='#003999', 
        fill='#FF580B').add_to(m)
    
m

KeyboardInterrupt: 

In [ ]:
#ccctv 위치 핀
cnt = 0
m = folium.Map(location=[cctv_loc_tm['gpsCrdntX'].mean(), cctv_loc_tm['gpsCrdntY'].mean() ], zoom_start=13)

for idx, row in cctv_loc_tm.iterrows():
    lat = row['gpsCrdntX']
    lng = row['gpsCrdntY']
    name = row['flStrePath'] if 'flStrePath' in cctv_loc_tm.columns else 'CCTV'
    cnt = cnt + 1
    print(cnt , lat,lng)
    folium.Marker(
        location = [lat, lng],
        popup=name,
        icon = folium.Icon(color='blue', icon='camera', prefix='fa')
    ).add_to(m)


m.save('cctv_map.html')

In [49]:
location_counts = (
    illigal_fixture.groupby(['latitude', 'longitude'])
    .size()
    .reset_index(name='count')
)

location_counts['log_count'] = np.log1p(location_counts['count'])

In [65]:
location_counts.describe()

,latitude,longitude,count,log_count
count,443.000000,443.000000,443.000000,443.000000
mean,37.273035,127.018154,806.609481,5.727571
std,0.019493,0.034521,1344.786137,1.531106
min,37.232960,126.934853,1.000000,0.693147
25%,37.258930,126.994949,107.500000,4.686740
50%,37.272597,127.019319,371.000000,5.918894
75%,37.289975,127.046903,916.000000,6.820974
max,37.334334,127.085776,11061.000000,9.311271


In [73]:
steps = 100
min_val = 1
max_val = 10000
color_map = cm.LinearColormap(
    colors=['blue','green','yellow','orange','red'],
    vmin=min_val,
    vmax=max_val
).to_step(n=10)

gradient_map = defaultdict(dict)
for i in range(steps):
    gradient_map[1/steps*i] = color_map.rgb_hex_str(1/steps*i)
gradient_map = {
    (i / steps): color_map.rgb_hex_str(min_val + (max_val - min_val) * i / steps)
    for i in range(steps + 1)
}

m = folium.Map(location=[lat,lng], tiles='openstreetmap', zoom_start=11)



HeatMap(
    data = location_counts[['latitude', 'longitude', 'count']].values,
    #data = illigal_fixture[['latitude','longitude']].values,
    radius=10,
    gradient=gradient_map
).add_to(m)

color_map.caption = 'Counts'
m.add_child(color_map)

m

### 주차단속 데이터 + 고정형 데이터 + 이동형 카메라 데이터

In [63]:
data_notcctv3 = pd.read_csv('./processed_data/data3_processed_data.csv')
data_notcctv1 = pd.read_csv('./processed_data/data1_processed_data.csv', encoding='euc-kr')
data_notcctv5 = pd.read_csv('./processed_data/data5_processed_data.csv')

In [64]:
data_notcctv = pd.concat([data_notcctv1, data_notcctv3, data_notcctv5])
data_notcctv.head(5)

,집계년도,시군명,관리기관명,단속일시정보,단속장소,단속방법,데이터기준일자,단속일시_월,단속일시_연,단속일시_일,단속일시_요일,단속일시_시,단속일시_isdaytime,단속일시_season,latitude,longitude
0,2022,수원시,권선구청,2022-01-01 00:33:00,권선동 1124,국민신문고,2023-12-01,1,2022,1,Saturday,0,night,winter,37.257077,127.026332
1,2022,수원시,권선구청,2022-01-01 08:48:00,세류동 1159-2,국민신문고,2023-12-01,1,2022,1,Saturday,8,morning,winter,37.243852,127.014963
2,2022,수원시,권선구청,2022-01-01 08:57:00,구운동 583,국민신문고,2023-12-01,1,2022,1,Saturday,8,morning,winter,37.276189,126.970282
3,2022,수원시,권선구청,2022-01-01 09:01:00,구운동 966,국민신문고,2023-12-01,1,2022,1,Saturday,9,morning,winter,37.277053,126.971851
4,2022,수원시,권선구청,2022-01-01 09:02:00,구운동 966,국민신문고,2023-12-01,1,2022,1,Saturday,9,morning,winter,37.277053,126.971851


In [65]:
data_notcctv.shape

(224376, 16)

In [66]:
data_notcctv['단속장소'].value_counts()

단속장소
신동 권선로882번길           2359
신동 신원로                2245
이의동 센트럴타운로            2102
금곡동 금곡로197번길          1755
고색동 산업로156번길          1676
                      ... 
영통구 영통동 994-2            1
인계동 90-2 (횡단보도)          1
서둔동 323-167              1
매산로1가 매산로1가 18           1
곡반정동 516-10 교차로모퉁이       1
Name: count, Length: 24536, dtype: int64

In [68]:
data_notcctv.isna().sum()

집계년도                  0
시군명                   0
관리기관명                 0
단속일시정보                0
단속장소                  0
단속방법                  0
데이터기준일자               0
단속일시_월                0
단속일시_연                0
단속일시_일                0
단속일시_요일               0
단속일시_시                0
단속일시_isdaytime        0
단속일시_season           0
latitude          12807
longitude         12807
dtype: int64

In [69]:
data_notcctv = (data_notcctv.dropna(axis=0))

In [70]:
data_notcctv.shape

(211569, 16)

In [71]:
data_notcctv

data_notcctv_agg=data_notcctv.groupby('단속장소').agg(
    단속장소=('단속장소', 'size'),
    평균위도=('latitude', 'mean'),
    평균경도=('longitude', 'mean')
)
data_notcctv_agg

,단속장소,평균위도,평균경도
단속장소,,,
경기도 수원시 권선구 효원로256번길 7,1,37.261649,127.029775
경기도 수원시 영통구 법조로 105,1,37.290493,127.069337
"경기도 수원시 팔달구 중부대로 67,",1,37.277175,127.024493
경기도수원시권선구고색동 수원시 권선구 고색동 1100,1,37.242239,126.987241
경기도수원시권선구고색동 수원시 권선구 고색동 1112,14,37.245766,126.984691
...,...,...,...
화서동 화서동726부근 (교차로 모퉁이 5m이내),1,37.285557,126.981350
화서동 화서동729부근 (교차로 모퉁이 5m이내),1,37.285856,126.982693
화서동 화서동730부근 (교차로 모퉁이 5m이내),1,37.284729,126.982222


In [72]:
data_notcctv = geo_transform(data_notcctv)

/opt/anaconda3/envs/data_venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/anaconda3/envs/data_venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [73]:
location_counts = (
    data_notcctv.groupby(['latitude', 'longitude'])
    .size()
    .reset_index(name='count')
)

location_counts['log_count'] = np.log1p(location_counts['count'])

In [74]:
location_counts

,latitude,longitude,count,log_count
0,35.787900,126.930693,1,0.693147
1,35.796817,129.325854,1,0.693147
2,35.803146,127.147755,2,1.098612
3,35.803214,127.147925,3,1.386294
4,35.814920,126.893182,1,0.693147
...,...,...,...,...
14059,37.643323,127.214849,17,2.890372
14060,37.887631,127.136189,1,0.693147
14061,37.888916,127.137769,1,0.693147
14062,37.891497,127.137067,1,0.693147


In [75]:
location_counts.describe()

,latitude,longitude,count,log_count
count,14064.000000,14064.000000,14064.000000,14064.000000
mean,37.274684,127.016178,15.043302,1.511507
std,0.049453,0.042965,74.763846,1.110597
min,35.787900,126.645442,1.000000,0.693147
25%,37.262430,127.000051,1.000000,0.693147
50%,37.273790,127.017125,2.000000,1.098612
75%,37.285861,127.031740,6.000000,1.945910
max,37.892145,129.325854,2359.000000,7.766417


In [ ]:
steps = 100
min_val = 1
max_val = 1000

color_map = cm.LinearColormap(
    colors=['blue','green','yellow','orange','red'],
    vmin=min_val,
    vmax=max_val
).to_step(n=10)

gradient_map = defaultdict(dict)
for i in range(steps):
    gradient_map[1/steps*i] = color_map.rgb_hex_str(1/steps*i)
gradient_map = {
    (i / steps): color_map.rgb_hex_str(min_val + (max_val - min_val) * i / steps)
    for i in range(steps + 1)
}

m = folium.Map(location=[lat,lng], tiles='openstreetmap', zoom_start=11)



HeatMap(
    data = location_counts[['latitude', 'longitude', 'count']].values,
    #data = illigal_fixture[['latitude','longitude']].values,
    radius=10,
    gradient=gradient_map
).add_to(m)

color_map.caption = 'Counts'
m.add_child(color_map)

m